In [1]:

from sklearn.model_selection import train_test_split

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

import numpy as np
from moabb.paradigms import MotorImagery
from sklearn import preprocessing
from braindecode.datasets import MOABBDataset


# import adabound

import torch
import torch.nn as nn 
from torch import tensor



In [2]:
from utils import utils
from utils import data_utils

In [3]:
utils.set_seed(0)
device = utils.set_device()

In [4]:
from moabb import datasets

In [5]:
# data config
dataset_name="BNCI2015001"
subject_id=1

paradigm = MotorImagery(n_classes=2)
dataset = getattr(datasets, "BNCI2015001")
subjects = [subject_id]

# load data
X, y, metadata = paradigm.get_data(dataset=dataset(), subjects=subjects)
X = X * 1000
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

# load metadata
ds = MOABBDataset(dataset_name=dataset_name, subject_ids=subject_id)
ch_names = ds.datasets[0].raw.info["ch_names"]
samplerate = ds.datasets[0].raw.info["sfreq"]



Choosing from all possible events


200 events found
Event IDs: [1 2]
200 events found
Event IDs: [1 2]


In [6]:
splitter = getattr(data_utils, "partition_"+dataset_name)
X_, X_test, y_, y_test = splitter(X, y, metadata)

In [7]:
import wandb

wandb.login()

wandb: Currently logged in as: mirfarid (use `wandb login --relogin` to force relogin)


True

In [8]:
from models.model import *
from models.loss import csp_loss

In [9]:
def objective(trial):
   
    #hyperparameters
    n_components = 2
    filters = 20
    component_order = 'alternate'
    
    

    
    # define optimizers
    
    optimizer_names = ['adam', 'momentum']
    optimizer_name_model = trial.suggest_categorical('optim_type_model', optimizer_names)
    weight_decay_model = trial.suggest_loguniform('weight_decay_model', 1e-8, 1e-3)
    lr_model = trial.suggest_loguniform('lr_model', 1e-5, 1e-1)


    optimizer_name_clf = trial.suggest_categorical('optim_type_clf', optimizer_names)
    weight_decay_clf = trial.suggest_loguniform('weight_decay_clf', 1e-8, 1e-3)
    lr_clf = trial.suggest_loguniform('lr_clf', 1e-5, 1e-1)


        
    # definde model and classifier
    clf = Classifier(n_components, filters).to(device)
    model = Tception(samplerate, filters).to(device)
    CSPloss = csp_loss(n_components = n_components,
                       n_channels = 13,
                       device=device,
                       component_order = component_order)

    criterion = nn.BCELoss()
    
    if optimizer_name_model == optimizer_names[0]:
        optimizer_model = torch.optim.Adam(model.parameters(),
                                     lr=lr_model,
                                     weight_decay=weight_decay_model,
                                     amsgrad=True)
    elif optimizer_name_model == optimizer_names[1]:
        optimizer_model = torch.optim.SGD(model.parameters(),
                                    lr=lr_model,
                                    momentum=0.9,
                                    weight_decay=weight_decay_model)

        
    if optimizer_name_clf == optimizer_names[0]:
        optimizer_clf = torch.optim.Adam(clf.parameters(),
                                     lr=lr_clf,
                                     weight_decay=weight_decay_clf,
                                     amsgrad=True)
    elif optimizer_name_clf == optimizer_names[1]:
        optimizer_clf = torch.optim.SGD(clf.parameters(),
                                    lr=lr_clf,
                                    momentum=0.9,
                                    weight_decay=weight_decay_clf)

    
    
    val_losses, accs = [], []   
    
    
    
    X_, X_test, y_, y_test = splitter(X, y, metadata)
    skf = StratifiedKFold(n_splits=5)
    for run_ind, (train_index, val_ind) in enumerate(skf.split(X_, y_)):

        X_train, X_valid = X[train_index], X[val_ind]
        y_train, y_valid = y[train_index], y[val_ind]
        
        X_test = X[metadata['session'] == 'session_B']
        y_test = y[metadata['session'] == 'session_B']
        
        config = {'n_components': n_components,
          'filters': filters,
          'component_order': component_order,
          'optim_type_model': optimizer_name_model,
          'weight_decay_model': weight_decay_model,
          'lr_model': lr_model,
          'optim_type_clf': optimizer_name_clf,
          'weight_decay_clf': weight_decay_clf,
          'lr_clf': lr_clf,
          'run': run_ind}

        X_train = torch.as_tensor(X_train).float().unsqueeze(1).to(device)
        y_train = torch.as_tensor(y_train).float().to(device)
        X_val = torch.as_tensor(X_valid).float().unsqueeze(1).to(device)
        y_val = torch.as_tensor(y_valid).float().to(device)
        X_test = torch.as_tensor(X_test).float().unsqueeze(1).to(device)
        y_test = torch.as_tensor(y_test).float().to(device)

        
        clf.reset_parameters()
        model.reset_parameters()

        for epoch in tqdm(range(50)):

            
            model.train()
            clf.train()

            optimizer_model.zero_grad()

            H = model(X_train)
            cpsloss_train , transformed = CSPloss(H, y_train)

            cpsloss_train.backward(retain_graph=True)
            optimizer_clf.zero_grad()

            outputs = clf(transformed.clone().detach())
            loss_train = criterion(outputs, y_train)


            loss_train.backward()
            optimizer_model.step()
            optimizer_clf.step()

            accuracy_train = ((outputs > 0.5) == y_train).float().mean()
            model.eval()
            clf.eval()

            with torch.no_grad():
                H = model(X_val)

                cpsloss_val, transformed = CSPloss.filter_bank_val(H, y_val)

                outputs = clf(transformed.clone().detach())
                loss_val = criterion(outputs, y_val)
                accuracy_val = ((outputs > 0.5) == y_val).float().mean()

                H = model(X_test)

                cpsloss_test, transformed = CSPloss.filter_bank_val(H, y_test)

                outputs = clf(transformed.clone().detach())
                loss_test = criterion(outputs, y_test)
                accuracy_test = ((outputs > 0.5) == y_test).float().mean()
            val_losses.append(loss_val)
            accs.append(accuracy_test)

    loss, acc = tensor(val_losses), tensor(accs)
    loss, acc = loss.view(5, 500), acc.view(5, 500)
    loss, argmin = loss.min(dim=1)
    acc = acc[torch.arange(5, dtype=torch.long), argmin]
    loss_mean = loss.mean().item()
    acc_mean = acc.mean().item()
    acc_std = acc.std().item()


    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()


    return acc_mean

In [9]:
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback

In [11]:
wandb_kwargs = {"project": "bci-first-test"}
wandbc = WeightsAndBiasesCallback(metric_name="5foldValidAccuracy", wandb_kwargs=wandb_kwargs)

study = optuna.create_study(direction="maximize", study_name="my_study")
study.optimize(objective, n_trials=10, callbacks=[wandbc])

/tmp/ipykernel_76746/1095988918.py:2: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(metric_name="5foldValidAccuracy", wandb_kwargs=wandb_kwargs)


[I 2022-01-03 22:57:24,684] A new study created in memory with name: my_study
  0%|          | 0/50 [00:00<?, ?it/s]/home/puppy/projects/BCI/models/loss.py:89: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).transpose(-2, -1).conj().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180487213/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:1285.)
  L = torch.cholesky(S_t, upper=False)
/home/puppy/projects/BCI/models/loss.py:90: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the

KeyboardInterrupt: 

In [10]:
from utils.model_utils import get_config, get_optim

In [11]:
trial_num = -1 # trial number of optuna

def objective(trial):
    global trial_num
    trial_num += 1
    #hyperparameters
    n_components = 2
    component_order = 'alternate'
    
    # get parameters specific to the current trial
    
    config = get_config(trial=trial)
        
    # definde model and classifier
    clf = Classifier(n_components, config['filters']).to(device)
    model = Tception(config.samplerate, config.['filters']).to(device)
    CSPloss = csp_loss(n_components = n_components,
                       n_channels = 13,
                       device=device,
                       component_order = component_order)

    # get optimizers specific to the current trial
    
    optimizer_model, optimizer_clf = get_optim(config, model, clf)
    criterion = nn.BCELoss()    

    params = {'n_components': n_components,
          'filters': config['filters'],
          'component_order': component_order,
          'optim_type_model': config['optimizer_name_model'],
          'weight_decay_model': config['weight_decay_model'],
          'lr_model': config['lr_model'],
          'optim_type_clf': config['optimizer_name_clf'],
          'weight_decay_clf': config['weight_decay_clf'],
          'lr_clf': config['lr_clf'],
          'trial_id': trial_num}

    wandb.config = params

    val_losses, accs = [], []   
       
    X_, X_test, y_, y_test = splitter(X, y, metadata)
    skf = StratifiedKFold(n_splits=5)
    for run_ind, (train_index, val_ind) in enumerate(skf.split(X_, y_)):

        X_train, X_valid = X[train_index], X[val_ind]
        y_train, y_valid = y[train_index], y[val_ind]
        
        X_test = X[metadata['session'] == 'session_B']
        y_test = y[metadata['session'] == 'session_B']
        
        

        X_train = torch.as_tensor(X_train).float().unsqueeze(1).to(device)
        y_train = torch.as_tensor(y_train).float().to(device)
        X_val = torch.as_tensor(X_valid).float().unsqueeze(1).to(device)
        y_val = torch.as_tensor(y_valid).float().to(device)
        
        clf.reset_parameters()
        model.reset_parameters()

        for epoch in tqdm(range(5)):

            
            model.train()
            clf.train()

            optimizer_model.zero_grad()

            H = model(X_train)
            cpsloss_train , transformed = CSPloss(H, y_train)

            cpsloss_train.backward(retain_graph=True)
            optimizer_clf.zero_grad()

            outputs = clf(transformed.clone().detach())
            loss_train = criterion(outputs, y_train)


            loss_train.backward()
            optimizer_model.step()
            optimizer_clf.step()

            accuracy_train = ((outputs > 0.5) == y_train).float().mean()
            model.eval()
            clf.eval()

            with torch.no_grad():
                H = model(X_val)

                cpsloss_val, transformed = CSPloss.filter_bank_val(H, y_val)

                outputs = clf(transformed.clone().detach())
                loss_val = criterion(outputs, y_val)
                accuracy_val = ((outputs > 0.5) == y_val).float().mean()

                H = model(X_test)

                cpsloss_test, transformed = CSPloss.filter_bank_val(H, y_test)

                outputs = clf(transformed.clone().detach())
                loss_test = criterion(outputs, y_test)
                accuracy_test = ((outputs > 0.5) == y_test).float().mean()
            val_losses.append(loss_val)
            accs.append(accuracy_test)
            wandb.log({
                'train_accuracy': accuracy_train,
                'train_loss': loss_train,
                'train_csploss': cpsloss_train,
                'valid_accuracy': accuracy_val,
                'valid_loss': loss_val,
                'valid_csploss': cpsloss_val,
                'run': run_ind,
                'epoch': epoch
            })
        

    loss, acc = tensor(val_losses), tensor(accs)
    loss, acc = loss.view(5, 500), acc.view(5, 500)
    loss, argmin = loss.min(dim=1)
    acc = acc[torch.arange(5, dtype=torch.long), argmin]
    loss_mean = loss.mean().item()
    acc_mean = acc.mean().item()
    acc_std = acc.std().item()
    wandb.log({
        'cv_acc_mean': acc_mean,
        'cv_acc_std': acc_std,
        'cv_loss_mean': loss_mean
        })

    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()


    return acc_mean

In [12]:
wandb_kwargs = {"project": "bci-second-test"}
wandbc = WeightsAndBiasesCallback(metric_name="5foldValidAccuracy", wandb_kwargs=wandb_kwargs)

study = optuna.create_study(direction="maximize", study_name="my_study")
study.optimize(objective, n_trials=10, callbacks=[wandbc])

/tmp/ipykernel_90663/1609232462.py:2: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(metric_name="5foldValidAccuracy", wandb_kwargs=wandb_kwargs)


[I 2022-01-04 02:05:49,937] A new study created in memory with name: my_study
[W 2022-01-04 02:05:49,951] Trial 0 failed because of the following error: AttributeError("'dict' object has no attribute 'filters'")
Traceback (most recent call last):
  File "/home/puppy/anaconda3/envs/bci/lib/python3.9/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_90663/1220566475.py", line 15, in objective
    clf = Classifier(n_components, config.filters).to(device)
AttributeError: 'dict' object has no attribute 'filters'


AttributeError: 'dict' object has no attribute 'filters'